## Notebook that creates and deploys ML Classification project using Azure ML, MLflow, Scikit-learn and Pandas

----------------------------------------------------------------------------------
- ##### Data accessed via external url
- ##### Gradient Boosting Classifier is used for classification task 
- ##### Script is not reusable since "Components" for the pipeline are not created 
----------------------------------------------------------------------------------

### Creating handle to workspace

In [1]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# authentication
credential = DefaultAzureCredential()
ml_client = MLClient.from_config(credential, path = ".config.json")

# verification
ws = ml_client.workspaces.get("alexlearn_ML_Studio")
print(ws.location,":", ws.resource_group)

Found the config file in: /mnt/batch/tasks/shared/LS_root/mounts/clusters/alexlearncompute/code/Users/alexey.kazakov/likelihood of credit card payment/config.json


westeurope : DSCGlobalCEEDTANLTst


### Creating a job environment

##### Creating directory fro storing configuration file

In [2]:
import os

dependencies_dir = "./dependencies"
os.makedirs(dependencies_dir, exist_ok=True)

##### Creating yaml file

In [3]:
%%writefile {dependencies_dir}/conda.yaml
name: model-env
channels:
  - conda-forge
dependencies:
  - python=3.8
  - numpy=1.21.2
  - pip=21.2.4
  - scikit-learn=1.0.2
  - scipy=1.7.1
  - pandas>=1.1,<1.2
  - pip:
    - inference-schema[numpy-support]==1.3.0
    - mlflow==2.8.0
    - mlflow-skinny==2.8.0
    - azureml-mlflow==1.51.0
    - psutil>=5.8,<5.9
    - tqdm>=4.59,<4.60
    - ipykernel~=6.0
    - matplotlib

Overwriting ./dependencies/conda.yaml


##### Referencing yaml file to create and register a custom environment

In [4]:
from azure.ai.ml.entities import Environment

custom_env_name = "aml-scikit-learn"

custom_job_env = Environment(
    name=custom_env_name,
    description="Custom environment for Credit Card Defaults job",
    tags={"scikit-learn": "1.0.2"},
    conda_file=os.path.join(dependencies_dir, "conda.yaml"),
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)
custom_job_env = ml_client.environments.create_or_update(custom_job_env)

print(
    f"Environment with name {custom_job_env.name} is registered to workspace, the environment version is {custom_job_env.version}"
)

Environment with name aml-scikit-learn is registered to workspace, the environment version is 2


### Creating training script

##### Creating source folder for the script

In [5]:
import os

train_src_dir = "./src"
os.makedirs(train_src_dir, exist_ok=True)

##### Creating training script in source folder

In [6]:
%%writefile {train_src_dir}/main.py
import os
import argparse
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

def main():
    """Main function of the script."""

    # input and output arguments
    parser = argparse.ArgumentParser()
    parser.add_argument("--data", type=str, help="path to input data")
    parser.add_argument("--test_train_ratio", type=float, required=False, default=0.25)
    parser.add_argument("--n_estimators", required=False, default=100, type=int)
    parser.add_argument("--learning_rate", required=False, default=0.1, type=float)
    parser.add_argument("--registered_model_name", type=str, help="model name")
    args = parser.parse_args()
   
    # Start Logging
    mlflow.start_run()

    # enable autologging
    mlflow.sklearn.autolog()

    ###################
    #<prepare the data>
    ###################
    print(" ".join(f"{key}={value}" for key, value in vars(args).items()))

    print("input data:", args.data)
    
    credit_df = pd.read_csv(args.data, header=1, index_col=0)

    mlflow.log_metric("num_samples", credit_df.shape[0])
    mlflow.log_metric("num_features", credit_df.shape[1] - 1)

    #Split train and test datasets
    train_df, test_df = train_test_split(
        credit_df,
        test_size=args.test_train_ratio,
    )
    ####################
    #</prepare the data>
    ####################

    ##################
    #<train the model>
    ##################
    # Extracting the label column
    y_train = train_df.pop("default payment next month")

    # convert the dataframe values to array
    X_train = train_df.values

    # Extracting the label column
    y_test = test_df.pop("default payment next month")

    # convert the dataframe values to array
    X_test = test_df.values

    print(f"Training with data of shape {X_train.shape}")

    clf = GradientBoostingClassifier(
        n_estimators=args.n_estimators, learning_rate=args.learning_rate
    )
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    print(classification_report(y_test, y_pred))
    ###################
    #</train the model>
    ###################

    ##########################
    #<save and register model>
    ##########################
    # Registering the model to the workspace
    print("Registering the model via MLFlow")
    mlflow.sklearn.log_model(
        sk_model=clf,
        registered_model_name=args.registered_model_name,
        artifact_path=args.registered_model_name,
    )

    # Saving the model to a file
    mlflow.sklearn.save_model(
        sk_model=clf,
        path=os.path.join(args.registered_model_name, "trained_model"),
    )
    ###########################
    #</save and register model>
    ###########################
    
    # Stop Logging
    mlflow.end_run()

if __name__ == "__main__":
    main()

Overwriting ./src/main.py


### Configuring the command

In [7]:
from azure.ai.ml import command
from azure.ai.ml import Input

registered_model_name = "credit_defaults_model"

job = command(
    inputs=dict(
        data=Input(
            type="uri_file",
            path="https://azuremlexamples.blob.core.windows.net/datasets/credit_card/default_of_credit_card_clients.csv",
        ),
        test_train_ratio=0.2,
        learning_rate=0.25,
        registered_model_name=registered_model_name,
    ),
    code="./src/",  # location of source code
    command="python main.py --data ${{inputs.data}} --test_train_ratio ${{inputs.test_train_ratio}} --learning_rate ${{inputs.learning_rate}} --registered_model_name ${{inputs.registered_model_name}}",
    environment="aml-scikit-learn@latest",
    display_name="credit_default_prediction",
)

### Submitting the job 

In [8]:
ml_client.create_or_update(job)

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Experiment,Name,Type,Status,Details Page
likelihoodofcreditcardpayment,tidy_comb_zq7kmgrq1l,command,Starting,Link to Azure Machine Learning studio


### Create an online Endpoint

In [9]:
import uuid

# Create a unique name for the endpoint
online_endpoint_name = "credit-endpoint-" + str(uuid.uuid4())[:8]

In [10]:
from azure.ai.ml.entities import ManagedOnlineEndpoint

# define an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="this is an online endpoint",
    auth_mode="key",
    tags={
        "training_dataset": "credit_defaults",
    },
)

In [11]:
# create the online endpoint
# expect the endpoint to take approximately 2 minutes.

endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint).result()

##### Checking availability of the checkpoint

In [12]:
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

print(
    f'Endpoint "{endpoint.name}" with provisioning state "{endpoint.provisioning_state}" is retrieved'
)

Endpoint "credit-endpoint-eea481bc" with provisioning state "Succeeded" is retrieved


### Deploy the model to the endpoint

In [13]:
registered_model_name = "credit_defaults_model"

# Let's pick the latest version of the model
latest_model_version = max(
    [int(m.version) for m in ml_client.models.list(name=registered_model_name)]
)

print(latest_model_version)

2


In [14]:
from azure.ai.ml.entities import ManagedOnlineDeployment

# Choose the latest version of the registered model for deployment
model = ml_client.models.get(name=registered_model_name, version=latest_model_version)

# define an online deployment
blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    instance_type="Standard_DS3_v2",
    instance_count=1,
)

In [15]:
# create the online deployment
blue_deployment = ml_client.online_deployments.begin_create_or_update(
    blue_deployment
).result()

# blue deployment takes 100% traffic
# expect the deployment to take approximately 8 to 10 minutes.
endpoint.traffic = {"blue": 100}
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

Check: endpoint credit-endpoint-eea481bc exists
Readonly attribute principal_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>
Readonly attribute tenant_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>


.......................................................................................

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://credit-endpoint-eea481bc.westeurope.inference.ml.azure.com/score', 'openapi_uri': 'https://credit-endpoint-eea481bc.westeurope.inference.ml.azure.com/swagger.json', 'name': 'credit-endpoint-eea481bc', 'description': 'this is an online endpoint', 'tags': {'training_dataset': 'credit_defaults'}, 'properties': {'createdBy': 'Alexey Kazakov (DHL Supply Chain)', 'createdAt': '2024-08-28T15:34:37.956189+0000', 'lastModifiedAt': '2024-08-28T15:34:37.956189+0000', 'azureml.onlineendpointid': '/subscriptions/8b20e38c-72fb-4030-a702-128e713479ca/resourcegroups/dscglobalceedtanltst/providers/microsoft.machinelearningservices/workspaces/alexlearn_ml_studio/onlineendpoints/credit-endpoint-eea481bc', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/8b20e38c-72fb-4030-a702-128e713479ca/providers/Microsoft.MachineLearningServices/locations/westeurope/mfeOperationsS

### Checking the status of the endpoint

In [16]:
# return an object that contains metadata for the endpoint
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

# print a selection of the endpoint's metadata
print(
    f"Name: {endpoint.name}\nStatus: {endpoint.provisioning_state}\nDescription: {endpoint.description}"
)

Name: credit-endpoint-eea481bc
Status: Succeeded
Description: this is an online endpoint


### Testing the endpoint with sample data

In [17]:
import os

# Create a directory to store the sample request file.
deploy_dir = "./deploy"
os.makedirs(deploy_dir, exist_ok=True)

In [18]:
%%writefile {deploy_dir}/sample-request.json
{
  "input_data": {
    "columns": [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22],
    "index": [0, 1],
    "data": [
            [20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0],
            [10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 10, 9, 8]
            ]
                }
}

Overwriting ./deploy/sample-request.json


In [19]:
# test the blue deployment with the sample data
ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="blue",
    request_file="./deploy/sample-request.json",
)

'[1, 0]'

##### Get logs of the deployment

In [20]:
logs = ml_client.online_deployments.get_logs(
    name="blue", endpoint_name=online_endpoint_name, lines=50
)
print(logs)

Instance status:
SystemSetup: Succeeded
UserContainerImagePull: Succeeded
ModelDownload: Succeeded
UserContainerStart: Succeeded

Container events:
Kind: Pod, Name: ReadinessProbeFailed, Type: Warning, Time: 2024-08-28T15:40:30.15498Z, Message: Readiness probe failed: HTTP probe failed with statuscode: 502
Kind: Pod, Name: LivenessProbeFailed, Type: Warning, Time: 2024-08-28T15:40:36.048998Z, Message: Liveness probe failed: HTTP probe failed with statuscode: 502
Kind: Pod, Name: ReadinessProbeFailed, Type: Warning, Time: 2024-08-28T15:40:40.155076Z, Message: Readiness probe failed: HTTP probe failed with statuscode: 502
Kind: Pod, Name: LivenessProbeFailed, Type: Warning, Time: 2024-08-28T15:40:46.049119Z, Message: Liveness probe failed: HTTP probe failed with statuscode: 502
Kind: Pod, Name: ReadinessProbeFailed, Type: Warning, Time: 2024-08-28T15:40:50.154899Z, Message: Readiness probe failed: HTTP probe failed with statuscode: 502
Kind: Pod, Name: LivenessProbeFailed, Type: Warning,

### Create a second deployment

In [21]:
# pick the model to deploy. Here we use the latest version of the registered model
model = ml_client.models.get(name=registered_model_name, version=latest_model_version)

green_deployment = ManagedOnlineDeployment(
    name="green",
    endpoint_name=online_endpoint_name,
    model=model,
    instance_type="Standard_F4s_v2",
    instance_count=1,
)

# create the online deployment
# expect the deployment to take approximately 8 to 10 minutes
green_deployment = ml_client.online_deployments.begin_create_or_update(
    green_deployment
).result()

Check: endpoint credit-endpoint-eea481bc exists


.........................................................................

### Scaling deployment to handle more traffic

In [22]:
# update definition of the deployment
green_deployment.instance_count = 2

# update the deployment
# expect the deployment to take approximately 8 to 10 minutes
ml_client.online_deployments.begin_create_or_update(green_deployment).result()

Check: endpoint credit-endpoint-eea481bc exists


.........................................................

ManagedOnlineDeployment({'private_network_connection': None, 'package_model': False, 'provisioning_state': 'Succeeded', 'endpoint_name': 'credit-endpoint-eea481bc', 'type': 'Managed', 'name': 'green', 'description': None, 'tags': {}, 'properties': {'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/8b20e38c-72fb-4030-a702-128e713479ca/providers/Microsoft.MachineLearningServices/locations/westeurope/mfeOperationsStatus/odidp:9d51182b-b7c7-49fd-9d62-553e1490a69c:6d68596f-4745-412f-bfd9-5d2e42f452a3?api-version=2023-04-01-preview'}, 'print_as_yaml': False, 'id': '/subscriptions/8b20e38c-72fb-4030-a702-128e713479ca/resourceGroups/dscglobalceedtanltst/providers/Microsoft.MachineLearningServices/workspaces/alexlearn_ML_Studio/onlineEndpoints/credit-endpoint-eea481bc/deployments/green', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/alexlearncompute/code/Users/alexey.kazakov/likelihood of credit card payment', 'creation_context': 

### Updating traffic allocation for deployments

In [23]:
endpoint.traffic = {"blue": 80, "green": 20}
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

Readonly attribute principal_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>
Readonly attribute tenant_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>


ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://credit-endpoint-eea481bc.westeurope.inference.ml.azure.com/score', 'openapi_uri': 'https://credit-endpoint-eea481bc.westeurope.inference.ml.azure.com/swagger.json', 'name': 'credit-endpoint-eea481bc', 'description': 'this is an online endpoint', 'tags': {'training_dataset': 'credit_defaults'}, 'properties': {'createdBy': 'Alexey Kazakov (DHL Supply Chain)', 'createdAt': '2024-08-28T15:34:37.956189+0000', 'lastModifiedAt': '2024-08-28T15:34:37.956189+0000', 'azureml.onlineendpointid': '/subscriptions/8b20e38c-72fb-4030-a702-128e713479ca/resourcegroups/dscglobalceedtanltst/providers/microsoft.machinelearningservices/workspaces/alexlearn_ml_studio/onlineendpoints/credit-endpoint-eea481bc', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/8b20e38c-72fb-4030-a702-128e713479ca/providers/Microsoft.MachineLearningServices/locations/westeurope/mfeOperationsS

##### Testing traffic allocation by invoking the endpoint several times

In [25]:
response_list = []
for i in range(30):
    response = ml_client.online_endpoints.invoke(
        endpoint_name=online_endpoint_name,
        request_file="./deploy/sample-request.json"
    )
    response_list.append(response)
print(response_list[-5:])

['[1, 0]', '[1, 0]', '[1, 0]', '[1, 0]', '[1, 0]']


In [26]:
logs = ml_client.online_deployments.get_logs(
    name="green", endpoint_name=online_endpoint_name, lines=50
)
print(logs)

Instance status:
SystemSetup: Succeeded
UserContainerImagePull: Succeeded
ModelDownload: Succeeded
UserContainerStart: Succeeded

Container events:
Kind: Pod, Name: Downloading, Type: Normal, Time: 2024-08-28T15:47:02.893118Z, Message: Start downloading models
Kind: Pod, Name: Pulling, Type: Normal, Time: 2024-08-28T15:47:03.225485Z, Message: Start pulling container image
Kind: Pod, Name: Pulled, Type: Normal, Time: 2024-08-28T15:47:39.031782Z, Message: Container image is pulled successfully
Kind: Pod, Name: Downloaded, Type: Normal, Time: 2024-08-28T15:47:39.031782Z, Message: Models are downloaded successfully
Kind: Pod, Name: Created, Type: Normal, Time: 2024-08-28T15:47:39.159697Z, Message: Created container inference-server
Kind: Pod, Name: Started, Type: Normal, Time: 2024-08-28T15:47:39.220683Z, Message: Started container inference-server
Kind: Pod, Name: ReadinessProbeFailed, Type: Warning, Time: 2024-08-28T15:47:55.907302Z, Message: Readiness probe failed: HTTP probe failed wit

### Send all traffic to the new deployment

In [27]:
endpoint.traffic = {"blue": 0, "green": 100}
ml_client.begin_create_or_update(endpoint).result()

Readonly attribute principal_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>
Readonly attribute tenant_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>


ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://credit-endpoint-eea481bc.westeurope.inference.ml.azure.com/score', 'openapi_uri': 'https://credit-endpoint-eea481bc.westeurope.inference.ml.azure.com/swagger.json', 'name': 'credit-endpoint-eea481bc', 'description': 'this is an online endpoint', 'tags': {'training_dataset': 'credit_defaults'}, 'properties': {'createdBy': 'Alexey Kazakov (DHL Supply Chain)', 'createdAt': '2024-08-28T15:34:37.956189+0000', 'lastModifiedAt': '2024-08-28T15:34:37.956189+0000', 'azureml.onlineendpointid': '/subscriptions/8b20e38c-72fb-4030-a702-128e713479ca/resourcegroups/dscglobalceedtanltst/providers/microsoft.machinelearningservices/workspaces/alexlearn_ml_studio/onlineendpoints/credit-endpoint-eea481bc', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/8b20e38c-72fb-4030-a702-128e713479ca/providers/Microsoft.MachineLearningServices/locations/westeurope/mfeOperationsS

### Delete the old deployment

In [28]:
ml_client.online_deployments.begin_delete(
    name="blue", endpoint_name=online_endpoint_name
).result()

### Clean up

In [29]:
ml_client.online_endpoints.begin_delete(name=online_endpoint_name).result()

.................................................................